#검증세트
이전에는 train 세트와 test 세트로 모델을 훈련시키고 평가했다.<br>
하지만 test세트로 평가하고 test세트에 부적합 했을 때, 다시 test세트를의 score를 높이려고 한다.<br>
이 말은 모델을 train 세트로 학습하고 test 세트로 맞추는 것이라고 할 수 있다.<br>
이런 문제를 해결 하기 위해서 기존 train, test set으로 split 하는 것에 ***검증 세트***를 추가해서 데이터를 나눌 것이다.

In [52]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [53]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [54]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, random_state=42, test_size=0.2)

In [55]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, random_state=42, test_size=0.2)

In [56]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [57]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


#교차 검증
많은 데이터로 훈련 시킬 수록 좋은 모델이 된다. 그렇다고 검증 세트를 너무 작게 떼어 내면 검증이 제대로 이뤄지지 않는다. 이럴 때, ***교차검증***을 이용하면 안정적인 검증 점수와 많은 데이터를 훈련에 사용할 수 있다.<BR>
sklearn에 cross_validate()라는 교차 검증 함수가 있다.<br>
사용 방법은 첫번째, 매개변수에 평가할 모델 객체, train_input, train_target
---
교차 검증은 검증 세트를 떼어 내는 평가하는 과정을 여러 번 반복한다. 그 다음 점수를 **평균**하여 최종 검증 점수를 얻는다. 일반적으로 5-폴드, 10-폴드<br>
ex) 3-fold coss validation<br>
훈련세트, 훈련세트, 검증세트<br>
훈련세트, 검증세트, 훈련세트<br>
검증세트, 훈련세트, 훈련세트<br>



In [58]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)
import numpy as np
print(np.mean(scores['test_score']))  # 교차 검증의 최종 점수

{'fit_time': array([0.00808716, 0.00721049, 0.00759554, 0.0078876 , 0.0078764 ]), 'score_time': array([0.00098252, 0.00072646, 0.00078297, 0.00089622, 0.00079536]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


### cross_validate
output<br>
dict
fit_time 모델을 훈련하는 시간<br>
score_time 모델을 검증하는 시간<br>
test_score 검증세트로 평가한 점수<br>

---
회귀 모델일 경우: KFold 분할기<br>
분류 모델일 경우: StratifiedKFold<br>

---
cross_validate는 훈련세트를 섞어 폴드를 나누지 않는다.(섞고 싶다면 cv = splitter 분할기 (KFold, StratifiedKFold)사용)<br>
분할기에 n_splits 매개변수에 n 값을 넣었다면 n-fold cross validation


In [59]:
# 위 코드와 동일
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [60]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
score = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.855300214703487


# ***최적의 모델을 위한 하이퍼파라미터 탐색***

#그리드 서치(GridSearch)
하이퍼파라미터 탐색과 교차 검증을 한번에 수행한다.(cross_validate() 함수를 호출할 필요 없음)<br>
sklearn.model_selection의 GridSearchCV 클래스<br>
ex) 아래 결정 트리 모델에서 min_impurity_decrease 매개 변수의 최적값을 찾는 예제

n_jobs = -1 병렬 실행에 사용할 CPU 코어 수를 지정 기본값 1, (-1: 모든 코어)<br>
GridSearchCV의 best_estimator_ 속성에 검증 점수가 가장 높은 모델이 저장되어 있음.


In [61]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [62]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [63]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [64]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [65]:
print(gs.cv_results_)
print(gs.cv_results_['mean_test_score'])

{'mean_fit_time': array([0.00976543, 0.00977612, 0.00958123, 0.00887637, 0.00703568]), 'std_fit_time': array([0.00053807, 0.00093611, 0.00170292, 0.00114445, 0.00019754]), 'mean_score_time': array([0.00112638, 0.00109973, 0.00102434, 0.0009542 , 0.00118308]), 'std_score_time': array([7.98394870e-05, 1.71158611e-04, 3.26257684e-05, 4.43521106e-05,
       3.92064037e-04]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]), 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]), 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216,

In [66]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [67]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
'max_depth': range(5, 20, 1),
'min_samples_split': range(2, 100, 10)}

In [68]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [69]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [70]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


#랜덤 서치(Random Search)
랜덤 서치는 매개변수의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달

In [71]:
from scipy.stats import uniform, randint
# 모두 주어진 볌위에서 고르게 값을 뽑는 클래스, '균등분포에서 샘플링한다'라고 말한다.
# uniform은 실수 값, randint 정수 값을 뽑는다.
rgen = randint(0, 10) 
print(rgen.rvs(10))  # 10개 뽑음

[2 2 1 8 9 7 8 9 9 5]


In [72]:
np.unique(rgen.rvs(1000), return_counts=True)  # 1000개 뽑음

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([104,  90, 118, 104, 112,  94,  82,  98, 102,  96]))

In [73]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.63197065, 0.28467127, 0.75672613, 0.47338857, 0.91557851,
       0.70683119, 0.78252246, 0.23105418, 0.357207  , 0.02652279])

In [74]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)}

In [75]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f452f031bd0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f452f031090>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f452f0f3a50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f452f031050>},
                   random_state=42)

In [76]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [77]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [78]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8615384615384616
